<a href="https://colab.research.google.com/github/yan-69/My-projects/blob/master/cat_vs_dog_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O /tmp/cats_and_dogs_filtered.zip

In [ ]:
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'

ziP = zipfile.ZipFile(local_zip,'r')

ziP.extractall('/tmp')
ziP.close()

In [ ]:
base_dir = '/tmp/cats_and_dogs_filtered/'

train_dir = os.path.join(base_dir , 'train')
validation_dir = os.path.join(base_dir , 'validation')

train_cats_dir = os.path.join(train_dir , 'cats')
train_dogs_dir = os.path.join(train_dir , 'dogs')

valid_cats_dir = os.path.join(validation_dir , 'cats')
valid_dogs_dir = os.path.join(validation_dir , 'dogs')

In [ ]:
print("total training cats images :" , len(os.listdir(train_cats_dir)))
print("total training dogs images :" , len(os.listdir(train_dogs_dir)))
print("total validation cats images :" , len(os.listdir(valid_cats_dir)))
print("total validaiton dogs images :" ,len(os.listdir(valid_dogs_dir)))

In [ ]:
%matplotlib inline

import matplotlib.image as pimg
import matplotlib.pyplot as plt

rows = 4
columns = 4

index  = 0

In [ ]:
train_cat_fnames = os.listdir( train_cats_dir )
train_dog_fnames = os.listdir( train_dogs_dir )

figure = plt.gcf()

figure.set_size_inches(4*rows , 4*columns)

index += 8

next_cat = [ os.path.join(train_cats_dir , fname)
                                  for fname in train_cat_fnames[index-8:index] 
            ]
next_dog = [os.path.join(train_dogs_dir , fname)
                                  for fname in train_dog_fnames[index-8:index]
            ]

for i,img_path in enumerate(next_cat+next_dog):
  s = plt.subplot(rows,columns, i+1)
  s.axis('Off')

  img = pimg.imread(img_path)
  plt.imshow(img)

  plt.plot()


In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
                             tf.keras.layers.Conv2D(16,(3,3) , activation = 'relu' , input_shape=(150,150,3)),
                             tf.keras.layers.MaxPooling2D(2,2),
                             tf.keras.layers.Conv2D(32 , (3,3) , activation = 'relu'),
                             tf.keras.layers.MaxPooling2D(2,2),
                             tf.keras.layers.Conv2D(64 , (3,3) , activation = 'relu'),
                             tf.keras.layers.MaxPooling2D(2,2),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(512 , activation = 'relu'),
                             tf.keras.layers.Dense(1 , activation = 'sigmoid')

])

model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile( optimizer = RMSprop(lr = 0.001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen =  ImageDataGenerator(rescale = 1/255)
validation_datagen =  ImageDataGenerator(rescale = 1/255)

train_generator = train_datagen.flow_from_directory(train_dir ,
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (150 , 150))
validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                              batch_size = 20,
                                                              class_mode = 'binary',
                                                              target_size = (150 , 150))

In [ ]:
history = model.fit(
    train_generator,
    validation_data = validation_generator,
    steps_per_epoch = 100,
    epochs = 15,
    validation_steps = 50,
    verbose = 2
)

In [ ]:
import numpy as np

from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path='/content/' + fn
  img=image.load_img(path, target_size=(150, 150))
  
  x=image.img_to_array(img)
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])
  
  classes = model.predict(images, batch_size=10)
  
  print(classes[0])
  
  if classes[0]>0:
    print(fn + " is a dog")
    
  else:
    print(fn + " is a cat")

In [ ]:
import numpy as np
import random
from   tensorflow.keras.preprocessing.image import img_to_array, load_img

# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model.layers[1:]]

#visualization_model = Model(img_input, successive_outputs)
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

# Let's prepare a random input image of a cat or dog from the training set.
cat_img_files = [os.path.join(train_cats_dir, f) for f in train_cat_fnames]
dog_img_files = [os.path.join(train_dogs_dir, f) for f in train_dog_fnames]

img_path = random.choice(cat_img_files + dog_img_files)
img = load_img(img_path, target_size=(150, 150))  # this is a PIL image

x   = img_to_array(img)                           # Numpy array with shape (150, 150, 3)
x   = x.reshape((1,) + x.shape)                   # Numpy array with shape (1, 150, 150, 3)

# Rescale by 1/255
x /= 255.0

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]

# -----------------------------------------------------------------------
# Now let's display our representations
# -----------------------------------------------------------------------
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  
  if len(feature_map.shape) == 4:
    
    #-------------------------------------------
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    #-------------------------------------------
    n_features = feature_map.shape[-1]  # number of features in the feature map
    size       = feature_map.shape[ 1]  # feature map shape (1, size, size, n_features)
    
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    
    #-------------------------------------------------
    # Postprocess the feature to be visually palatable
    #-------------------------------------------------
    for i in range(n_features):
      x  = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std ()
      x *=  64
      x += 128
      x  = np.clip(x, 0, 255).astype('uint8')
      display_grid[:, i * size : (i + 1) * size] = x # Tile each filter into a horizontal grid

    #-----------------
    # Display the grid
    #-----------------

    scale = 20. / n_features
    plt.figure( figsize=(scale * n_features, scale) )
    plt.title ( layer_name )
    plt.grid  ( False )
    plt.imshow( display_grid, aspect='auto', cmap='viridis' ) 